<img src="./figs/IOAI-Logo.png" alt="IOAI Logo" width="200" height="auto">

[IOAI 2025 (Beijing, China), Individual Contest](https://ioai-official.org/china-2025)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/IOAI-official/IOAI-2025/blob/main/Individual-Contest/Antique/Solution/Antique_Solution.ipynb)

# Antique Painting Authentication: Reference Solution

## Step 1: Train Your Model

In [ ]:
import os
import sys

# 1. Get the current working directory
current_dir = os.getcwd()

# 2. Check if the path contains "Individual-Contest/Antique" and trim it to that point
if "Individual-Contest/Antique" in current_dir:
    root_index = current_dir.index("Individual-Contest/Antique") + len("Individual-Contest/Antique")
    project_root = current_dir[:root_index]
else:
    raise Exception("Project root directory not found. Please check the folder structure.")

# 3. Change working directory to the project root
os.chdir(project_root)
print("Working directory set to:", os.getcwd())

# 4. Add module search path (e.g., where metrics.py is located)
sys.path.append(os.path.join(project_root, "Scoring"))

In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import SpectralClustering
from collections import Counter
from sklearn.svm import SVC
import os

TRAIN_PATH = "./training_set/" # The address of trainig set

train = pd.read_csv(TRAIN_PATH + "training_set.csv")

X = np.array(train.iloc[:,:5])
y = np.array(train.iloc[:,5])

labeled_mask = y != 0
unlabeled_mask = y == 0
X_labeled = X[labeled_mask]
y_labeled = y[labeled_mask]
X_unlabeled = X[unlabeled_mask]

n_clusters = 2
spectral = SpectralClustering(n_clusters=n_clusters, affinity='rbf', gamma=10, random_state=42)
cluster_labels = spectral.fit_predict(X) 

cluster_to_label = {}
for cluster in range(n_clusters):

    labeled_in_cluster = y_labeled[cluster_labels[labeled_mask] == cluster]

    if len(labeled_in_cluster) > 0:
        most_common_label = Counter(labeled_in_cluster).most_common(1)[0][0]
        cluster_to_label[cluster] = most_common_label

pseudo_labels = np.array([cluster_to_label[cluster] for cluster in cluster_labels])

svm = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)
svm.fit(X, pseudo_labels)

## Step 2: Make Predictions on the Validation and Test Set

In [ ]:
VAL_DATA_PATH = "./Solution/validation_set/"
TEST_DATA_PATH = "./Solution/test_set/"

testA = np.array(pd.read_csv(VAL_DATA_PATH + "validation_set.csv"))
testB = np.array(pd.read_csv(TEST_DATA_PATH + "test_set.csv"))

predA = svm.predict(testA)
predB = svm.predict(testB)

## Step 3: Generate `submission.zip` for Submission

In [ ]:
import zipfile
import os

submissionA = pd.DataFrame(predA)
submissionA.to_csv("./Scoring/submissionA.csv", index=False, header=False)

submissionB = pd.DataFrame(predB)
submissionB.to_csv("./Scoring/submissionB.csv", index=False, header=False)

files_to_zip = ['./Scoring/submissionA.csv', './Scoring/submissionB.csv']
zip_filename = './Scoring/submission.zip'

with zipfile.ZipFile(zip_filename, 'w') as zipf:
    for file in files_to_zip:
        zipf.write(file, os.path.basename(file))

print(f'{zip_filename} is created succefully!')

### Evaluate the Model Performance

In [ ]:
%run Scoring/metrics.py